In [2]:
#### Vader compound score, input is a single review, to be applied as lambda
def vader_compound(review):
    from nltk import sent_tokenize
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()
    sentences = sent_tokenize(review)
    compound = 0
    for sentence in sentences:
        vs = analyzer.polarity_scores(sentence)
        compound+=vs['compound']/(len(sentences))
    return compound

##### EMOTION ANALYSIS ########
def get_nrc_data():
    nrc = "NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict

emotion_dict = get_nrc_data()
#where text is a single Amazon review
def emotion_analyzer(text,emotion_dict=emotion_dict):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0
    #Analyze the text and normalize by total number of words
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

In [3]:
#### REVIEW FILES/DATAFRAMES #
def parse(path):
    import gzip
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

# CONVERT TO DATE OBJECT
def str_dt(s):
    import datetime as dt
    # s_format = ‘06 2, 2013’
    s = s.split(' ') # [m,d,y]
    s[1] = s[1].strip(',')
    s = dt.date(int(s[2]),int(s[0]),int(s[1]))
    return s

def getDF(path):
    import pandas as pd
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    df = pd.DataFrame.from_dict(df, orient='index')
    df['reviewTime'] = df['reviewTime'].apply(lambda x: str_dt(x))
    del df['unixReviewTime']
    return df

In [4]:
beauty = getDF('reviews_Beauty_5.json.gz')

In [5]:
#### CLEANING TEXT ###########
def clean_txt(text):
    text = text.split()
    text = map(lambda x: clean_word(x), text)
    return ' '.join(text)

def clean_word(word):
    word = word.lower()
    for i in range(len(word)):
        if not (word[i].isalpha()) and word[i] != '-' and word[i] != "'":
            word = word[0:i]
            break
    return word

In [8]:
beauty['cleanText'] = beauty['reviewText'].apply(lambda x: clean_txt(x))

In [7]:
beauty['emotions'] = beauty['cleanText'].apply(lambda x: vader_compound(x))

KeyboardInterrupt: 

In [ ]:
beauty[['emotions']]